# Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os

In [ ]:
masechtot=[]
with open('/Users/ephraimmeiri/gitEtc/Rashi Thesis work/references subproject/input/שמות מסכתות.txt') as f:
    data = f.read()
    masechtot = data.split(', ')
    
masechtot_bavli =[]
with open('/Users/ephraimmeiri/gitEtc/Rashi Thesis work/references subproject/input/שמות מסכתות בבלי.txt') as f:
    data = f.read()
    masechtot_bavli = data.split(', ')

perek_names = []
with open('/Users/ephraimmeiri/gitEtc/Rashi Thesis work/references subproject/input/perakim_names1.txt') as f:
    data = f.read()
    perek_names = data.split(', ')
for name in perek_names:
    if len(name.split())>1:     
        if name.split()[0] == "רבי":
            perek_names.append("ר' ".join(name.split()[1:]))
    if "\"" in name:
        perek_names.append(name.replace("\"","\\\""))
perek_names_bavli = []
with open('/Users/ephraimmeiri/gitEtc/Rashi Thesis work/references subproject/input/perakim_names_Bavli.txt') as f:
    data = f.read()
    perek_names_bavli = data.split(', ')
    
pirkei_mishna=[]
with open("/Users/ephraimmeiri/gitEtc/Rashi Thesis work/references subproject/input/perakim_names_MishnaOnly.txt",'r') as f:
    data = f.read()
    pirkei_mishna = data.split(', ')
    
numbers = ["קמא","אחרון","בתרא","עשירי","תשיעי","שמיני","שביעי","ששי","חמישי","דביעי","שלישי","שני","ראשון"]
for letter in ["י","ט","ח","ז","ו","ה","ד","ג","ב","א"]:
        numbers.append(letter+"'")
        numbers.append("י\""+letter)
        numbers.append("כ\""+letter)
rashi_dir= '/Users/ephraimmeiri/gitEtc/Rashi Thesis work/Rashi plaintext/json filtered'

In [88]:
filtered_masechtot = masechtot.copy() # Names that are likely to be false positives
filtered_masechtot.remove("כלים")
filtered_masechtot.remove("ידים")

In [1]:
def get_rashi_file(masechet):
    masechet = masechet.replace("%20"," ")
    file = rashi_dir+"/Rashi on "+masechet+".json"
    if os.path.exists(file):
        with open(file) as f:
            data = json.load(f)
            return data
    else:
        return None

In [4]:
rashi_masechtot = ['Arakhin', 'Bekhorot', 'Chullin', 'Keritot', 'Menachot', 'Temurah', 'Zevachim', 'Beitzah', 'Chagigah', 'Eruvin', 'Pesachim', 'Megillah', 'Rosh Hashanah', 'Shabbat', 'Sukkah', 'Yoma', 'Gittin', 'Ketubot', 'Kiddushin', 'Sotah', 'Yevamot', 'Bava Batra', 'Bava Kamma', 'Bava Metzia', 'Avodah Zarah', 'Makkot', 'Sanhedrin', 'Shevuot', 'Niddah', 'Berakhot']

# Asking the questions

Queries:
דמסכת
במסכת <שם
בפרק <שם
בפרק N
בפרק <?> דמסכת
במסכת <?> בפרק	

In [ ]:
def get_results(query):
    results = []
    for file in os.listdir(rashi_dir):
        if file == '.DS_Store.json':
            continue
        name = file.split(' - ')[0][9:-5]
        print(name,end='\t')
        with open(rashi_dir+"/"+file) as f:
            data = json.load(f)
            for daf,d in zip(data['text'],range(len(data['text']))):
                for line,l in zip(daf,range(len(daf))):
                    for comment,c in zip(line,range(len(line))):
                        cont = query(comment)
                        if cont:
                            results.append([name,d,c,cont,comment])
    return results

In [7]:
def contains_masechet(text):
    results= None
    for masechet in masechtot_bavli:
        if "ב"+masechet in text:
            if results:
                results+=", "
                # print("Multi")
            else:
                results=""
            results+=masechet
    return results

# Right neighbors, for filtering

In [53]:
def get_right_neighbors(query,dist):
    results = []
    for name in rashi_masechtot:
        file = get_rashi_file(name)
        if file:
            for d in range(len(file['text'])):
                for l in range(len(file['text'][d])):
                    for c in range(len(file['text'][d][l])):
                        cont = query(file['text'][d][l][c])
                        if cont:
                            for mas in cont.split(', '):
                                strs= file['text'][d][l][c].split(mas) 
                                words = file['text'][d][l][c].split()
                                words1= [strs[0].split(),strs[1].split()]
                                if len(words1[1])>1 and len(words1[0])>1 and (words[len(words)-1] != words1[1][0]) :
                                    if words1[1]:  # Check if words1[1] is not empty
                                        words1[1][0] = words1[0][-1].join(strs[1][0]) if words1[0] else ""  # Check if words1[0] is not empty before accessing its last element
                                    if words1[0]:  # Check if words1[0] is not empty
                                        words1[0] = words1[0][:-1]
                                if len(words1[0])>dist:
                                    results.append("".join(words1[0][-dist:]))
    # map to frequency
    res_set = set(results)
    freq = {}
    for res in res_set:
        freq[res] = results.count(res)
    freq = sorted(freq.items(), key=lambda x: x[1], reverse=True)
    return  freq

In [54]:
mas_right_results = get_right_neighbors(contains_masechet,1)
print(len(mas_right_results))
mas_right_results

1229


[('ב', 332),
 ('-', 272),
 ('כדאמרינן', 263),
 ('כדאמר', 129),
 ('לן', 69),
 ("כדאמרי'", 65),
 ('מפרש', 59),
 ('דאמר', 55),
 ('לה', 51),
 ('אמרינן', 47),
 ('היא', 47),
 ('כדתנן', 47),
 ('דתנן', 40),
 ('דאמרינן', 30),
 ('ליה', 25),
 ('להיות', 23),
 ('באחד', 22),
 ('כדתניא', 22),
 ('ואמרינן', 21),
 ('דתניא', 18),
 ('אלא', 17),
 ('כדילפינן', 16),
 ('דקי"ל', 14),
 ('מפורש', 14),
 ('וב', 14),
 ('ואפילו', 13),
 ('וחבירו', 13),
 ('אבל', 13),
 ('הוא', 13),
 ('כדמפרש', 13),
 ('אפילו', 12),
 ('שלישי', 12),
 ('לו', 12),
 ('ודרשינן', 11),
 ('מר', 10),
 ('התם', 10),
 ('–', 10),
 ("ואמרי'", 10),
 ('תנן', 10),
 ("אמרי'", 10),
 ('שני', 10),
 ('בין', 9),
 ('להו', 9),
 ("דאמרי'", 8),
 ('ותניא', 8),
 ('דכתיב', 8),
 ('כוכבים', 8),
 ('כדאמרן', 7),
 ('מלאכה', 7),
 ('אחד', 7),
 ('ששנינו', 7),
 ('עליה', 7),
 ('נמי', 7),
 ('אף', 7),
 ('שמעתי', 7),
 ('טעמא', 6),
 ('תרומה', 6),
 ('למ"ד', 6),
 ('מים', 6),
 ('בשני', 6),
 ('כן', 6),
 ('ותנן', 6),
 ('הלכך', 6),
 ('בהן', 6),
 ('קי"ל', 6),
 ('שב', 6),
 ('הכא', 6),
 ('

Ok not actually that useful, too many words and not enough context. We'll have to settle for this level of messiness

In [24]:
mas_right_results = get_right_neighbors(contains_masechet,2)
print(len(mas_right_results))
mas_right_results

3653


['מפרש',
 'ואשתו',
 'כדאמר',
 'דעני',
 'ליתיה',
 'שאינו',
 'פחות',
 'פחות',
 'מנדר',
 'נמי',
 'שישנו',
 'דאיתיה',
 "דאמרי'",
 '-',
 'מפורש',
 'כדאמר',
 'גדול',
 'באחד',
 'נאפין',
 'של',
 'בתחילת',
 'השנה',
 "בא'",
 'כדאמרינן',
 'אחד',
 'המפורש',
 "כדאמרי'",
 'דאמר',
 'דאחד',
 'כדאמר',
 'באחד',
 "ואמרי'",
 'דכי',
 'כדאמר',
 'מפורש',
 'ותניא',
 "ואמרי'",
 '-',
 'מפרש',
 '-',
 'לן',
 'נידון',
 'אף',
 'חומר',
 'עלי',
 'אף',
 'דהואיל',
 'דכתיב',
 'דכתיב',
 '-',
 'חומר',
 'בנדרים',
 'דתניא',
 'משני',
 'ליה',
 'שלי',
 "דאמרי'",
 'כדאמרינן',
 'הגוזל',
 'יובל',
 'אמרינן',
 'כדתניא',
 'מפרש',
 'כדאמר',
 'בהגוזל',
 'מלאכה',
 'שובתת',
 'והרוכב',
 'מלאכה',
 'קדושה',
 'בנו',
 'ואמר',
 'כדאמר',
 'יהודה',
 'היא',
 'בפרשה',
 'אין',
 'אסור',
 'נוהג',
 'כדמפרש',
 'כדתנן',
 'מקדש',
 '-',
 'התפיסן',
 'מפרש',
 '-',
 'כדמפרש',
 'כהן',
 'ודכוותה',
 'כדאמרינן',
 'מפרש',
 'וחבירו',
 '-',
 'בדיבורו',
 'דאמרינן',
 'דאמרינן',
 '-',
 'טומאה.',
 'כדאמר',
 'ואפילו',
 '-',
 'מודו',
 'הכא',
 'אחוריו',
 'הנטמא',
 '-',
 

# Left neighbors -- do we have all the perek names?

In [ ]:
perek_names_one = []
for name in perek_names:
    if len(name.split())>1:     
        perek_names_one.append(name.split()[0])
        print(name.split()[0])
    else:
        perek_names_one.append(name)
        print("\t|"+name)

In [ ]:
def get_left_perek_neighbors(dist):
    results = []
    for name in rashi_masechtot:
        file = get_rashi_file(name)
        if file:
            for d in range(len(file['text'])):
                for l in range(len(file['text'][d])):
                    for c in range(len(file['text'][d][l])):
                        # cont = query(file['text'][d][l][c])
                        if "בפרק " in  file['text'][d][l][c]:
                            strs= file['text'][d][l][c].split("בפרק") 
                            words = file['text'][d][l][c].split()
                            words1= [strs[0].split(),strs[1].split()]
                            # if len(words1[1])>1 and len(words1[0])>1 and (words[len(words)-1] != words1[1][0]) :
                            #     if words1[1]:  # Check if words1[1] is not empty
                            #         words1[1][0] = words1[0][-1].join(strs[1][0]) if words1[0] else ""  # Check if words1[0] is not empty before accessing its last element
                            #     if words1[0]:  # Check if words1[0] is not empty
                            #         words1[0] = words1[0][:-1]
                            if len(words1[1])>dist:
                                neighbor = "".join(words1[1][:dist])
                                if neighbor not in perek_names_one and neighbor not in numbers:
                                    results.append("".join(words1[1][:dist]))
    # map to frequency
    res_set = set(results)
    freq = {}
    for res in res_set:
        freq[res] = results.count(res)
    freq = sorted(freq.items(), key=lambda x: x[1], reverse=True)
    return  freq

In [ ]:
perek_left_results = get_left_perek_neighbors(1)

In [128]:
for name in perek_names:
    if "\"" in name:
        print(name)
print("----------------")
for name in perek_names_one:
    if "\"" in name:
        print(name)

ב‪"‬ש
ר‪"‬ע
י‪"‬ט
יוה"כ
ר׳ גמליאל‪,‬ ר‪"‬ג
אע‪"‬פ
ר‪"‬ח
ר‪"‬א‪,‬שבעת ימים
ר‪"‬י
----------------
ב‪"‬ש
ר‪"‬ע
י‪"‬ט
יוה"כ
אע‪"‬פ
ר‪"‬ח
ר‪"‬א‪,‬שבעת
ר‪"‬י


In [129]:
for name in perek_names:
    if "בנות" in name:
        print(name)
print("----------------")
for name in perek_names_one:
    if "בנות" in name:
        print(name)

מלבנות התבואה
בנות שוח
כל קרבנות הצבור
יש בקרבנות
בנות כותים
----------------
מלבנות
בנות
בנות


In [130]:
perek_left_results

[('ב"ש', 20),
 ("ר'", 9),
 ('אע"פ', 6),
 ('ר"ג', 3),
 ('דלקמן', 2),
 ('ר"ע', 2),
 ('ר"א', 2),
 ('ין', 2),
 ('נזיר', 2),
 ('דלעיל', 2),
 ('מת', 1),
 ('(בנות', 1),
 ('השלישי', 1),
 ('בתרא)', 1),
 ('[יוצא', 1),
 ('הזה', 1),
 ('[בית', 1),
 ('מ"ה', 1),
 ('שלמעלה', 1),
 ('[כירה]', 1),
 ('[זה', 1),
 ('אמוראי', 1),
 ('[המפקיד]', 1),
 ('[בהמה', 1),
 ('שני)', 1),
 ('[ר"י]', 1),
 ('דגיד', 1),
 ('האשה.', 1),
 ('[ראשון]', 1),
 ('אחר', 1)]

Odd that some are showing up here, like ב"ש, אע"פ, and בנות כותים.

In [140]:
def get_results(query):
    results = []
    for file in os.listdir(rashi_dir):
        if file == '.DS_Store.json':
            continue
        name = file.split(' - ')[0][9:-5]
        print(name,end='\t')
        with open(rashi_dir+"/"+file) as f:
            data = json.load(f)
            for daf,d in zip(data['text'],range(len(data['text']))):
                for line,l in zip(daf,range(len(daf))):
                    for comment,c in zip(line,range(len(line))):
                        cont = query(comment)
                        if cont:
                            results.append([name,d,c,cont,comment])
    return results
def contains_perek(text):
    for name in perek_names:
        if "בפרק "+name in text:
            return name

In [141]:
perek_results = get_results(contains_perek)
perek_results_df = pd.DataFrame(perek_results,columns=["Masechet","Daf","Line","Perek","Comment"])

Bava Kamma	Sanhedrin	Bava Batra	Avodah Zarah	Makkot	Yevamot	Kiddushin	Niddah	Pesachim	Menachot	Sukkah	Yoma	Sotah	Shevuot	Temurah	Gittin	Bekhorot	Berakhot	Arakhin	Chullin	Rosh Hashanah	Ketubot	Zevachim	Megillah	Beitzah	Shabbat	Eruvin	Bava Metzia	Chagigah	Keritot	

In [142]:
perek_results_df.to_csv("perek_results1.csv")

# Defining the queries

In [55]:
def contains_masechet(text):
    results= None
    for masechet in masechtot_bavli:
        if "ב"+masechet in text or "במסכת "+masechet in text:
            if results:
                results+=", "
                # print("Multi")
            else:
                results=""
            results+=masechet
    return results

def contains_number(text):
    for num in numbers:
        if "בפרק "+num in text:
            return "בפרק "+num
    return None
def contains_perek(text):
    for name in perek_names:
        if "ב"+name in text:
            return name
def contains_masechet_bPerek(text):
    results= None
    for masechet in masechtot_bavli:
        if masechet+" בפרק" in text:
            if results:
                results+=", "
                # print("Multi")
            else:
                results=""    
            results+=masechet
        else:
            for name in perek_names:
                if masechet+"ב"+name in text:
                    if results:
                        results+=", "
                        # print("Multi")
                    else:
                        results=""
                    results+=  masechet+"ב"+name
    return results
def contains_perek_Dmasechet2(text):
    results= None
    for masechet in masechtot_bavli:
        if "ד"+masechet in text and "פרק" in text:
            if results:
                results+=", "
                # print("Multi")
            else:
                results=""
            results+=masechet
    return results

# Results